### Mapping waterlogged areas over the monsoon season

The monsoon season brings the majority of the annual rainfall to the wet tropical regions of Asia. In a normal year in  Bangladesh, nearly 20% of the country will be underwater after heavy rains and snowmelt cause coastal regions, rivers, and other areas to flood during the June to October monsoon season. 

To examine the changes in water occurance over the region, we'll use both multispectral and Synthetic Aperture Radar (SAR) data available through the DL Platform. Our goal is to produce a monthly snapshot of water coverage. Using a simple classification method to identify water, we'll produce a water product and upload it to our Platform.

In [1]:
import descarteslabs as dl

import numpy as np
import json
# import cartopy
# import shapely
# import shapely.geometry
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
import pickle
import calendar
import sklearn
import sklearn.cluster
import sklearn.ensemble
# from descarteslabs.client.services.catalog import catalog

# product_id = '6298b97d846a85f9045e8173d1f052c571b48cae:demo:water:bangladesh:v1'

## Prototyping

We'll start by prototyping and examining the data over a smaller region before scaling it to the entire country. 

In [2]:
country = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          103.5791015625,
          1.1342642839220822
        ],
        [
          104.13940429687499,
          1.1342642839220822
        ],
        [
          104.13940429687499,
          1.4884800029826135
        ],
        [
          103.5791015625,
          1.4884800029826135
        ],
        [
          103.5791015625,
          1.1342642839220822
        ]
      ]
    ]
  }
}


### Find imagery

In selecting classification features to identify water, we will state several assumptions about the spectral response of water relative to other land cover types:

* We expect backscatter over standing water and submerged areas from SAR to be very low in all polarizations.
* We expect NDVI to be low relative to vegetated areas.
* We expect NDWI to be high relative to unsubmerged areas.

In [3]:
lat, lon = 1.33883688455388, 103.8369369506836
          

In [4]:
tile = dl.raster.dltile_from_latlon(lat, 
                                    lon, 
                                    resolution = 20.0,
                                    tilesize = 1024,
                                    pad = 0)

In [ ]:
# available_scenes = dl.metadata.search(products=['sentinel-2:L1C'],
#                                       dltile=tile,
#                                       start_time='2017-01-01',
#                                       end_time='2017-12-31')

In [5]:
available_scenes, sent_2_ctx = dl.scenes.search(tile['geometry'],
                               products=['sentinel-2:L1C'],
                               start_datetime='2017-01-01',
                               end_datetime='2017-12-31')

In [6]:
print('Total number of scenes over ROI: {:>10}'.format(len(available_scenes)))

Total number of scenes over ROI:         49


### Develop classification model - Python

##### Define classification features to calculate from the imagery

In [7]:
product_dic = {'sentinel-1:GRD': {'bands': ['vv', 'vh'], 'stats': 'min'},
              'sentinel-2:L1C': {'bands': ['ndvi', 'ndwi', 'cloud-mask'], 'stats': 'max'}}

##### Calculate and visualize a sample of the data

In [8]:
date = ['2017-01-01', '2017-01-31']

all_stats = []   
for product in sorted(product_dic.keys()):
    print(product)
    ''' DL Metadata search for imagery over our AOI'''
#     available_scenes = dl.metadata.search(products = [product],
#                                           dltile = tile,
#                                           start_time = date[0],
#                                           end_time = date[1])
    available_scenes, ctx = dl.scenes.search(tile['geometry'],
                               products=[product],
                               start_datetime=date[0],
                               end_datetime=date[1])

#     ids = [scene['id'] for scene in available_scenes['features']]
    
    bands = product_dic[product]['bands']
    
    image_stack = None
    print(len(available_scenes))
    if product == 'sentinel-1:GRD':
        for scene in available_scenes:
            image, meta = scene.ndarray("vv vh", ctx=ctx, mask_alpha=False)
            print(np.unique(meta, return_counts=True))
            print(meta)
                  
    
        for scene in available_scenes:
            print(scene.ndarray("ndvi ndwi cloud-mask", ctx=ctx))
#             print(scene.ndarray(bands="derived:ndvi ndwi cloud-mask", ctx=ctx))else: 
            

#     for i, idd in enumerate(ids):

#         ''' DL Raster search to obtain image data '''
        
#         image, meta = dl.raster.ndarray(idd,
#                                         bands = bands,
#                                         dltile = tile)
    

#         if image_stack is None:

#             rx = image.shape[0]
#             ry = image.shape[1]
#             n_images = len(ids)
#             n_bands = 2
#             image_stack = np.zeros((n_images, rx, ry, n_bands))

#         ''' Mask the clouds '''
#         if "cloud-mask" in bands:
#             cloud_mask = image[:, :, 2]
#             image[cloud_mask == 1] = 0
#             image_stack[i, :, :, :] = image[:, :, :2]
            
#         else:
#             image_stack[i, :, :, :] = image

    
#     if product_dic[product]['stats'] == 'min':
#         ma = np.ma.masked_equal(image_stack, 0, copy = False)
#         all_stats.append(np.ma.min(ma, axis=0))
        
#     else:
#         all_stats.append(np.max(image_stack, axis = 0)[:, :, :2])

sentinel-1:GRD
3
(masked_array(data=[2, 3, 4, ..., --, 255, --],
             mask=[False, False, False, ...,  True, False,  True],
       fill_value=999999,
            dtype=uint8), array([  2,  64, 452, ...,   2, 212,   1]))
[[-- 43 36 ... 100 73 --]
 [-- 27 38 ... 61 49 --]
 [-- 26 26 ... 60 123 --]
 ...
 [-- 79 50 ... 75 70 --]
 [-- 27 33 ... 255 255 --]
 [-- 22 32 ... 255 247 --]]
(masked_array(data=[--],
             mask=[ True],
       fill_value=999999,
            dtype=uint8), array([1050624]))
[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]]
(masked_array(data=[2, 3, 4, ..., --, 255, --],
             mask=[False, False, False, ...,  True, False,  True],
       fill_value=999999,
            dtype=uint8), array([  2,  39, 350, ...,   2,  49,   1]))
[[-- 65 83 ... 71 54 --]
 [-- 30 30 ... 59 36 --]
 [-- 70 109 ... 40 126 --]
 ...
 [-- 92 91 ... 58 60 --]
 [-- 57 34 ...

ValueError: Band 'ndvi' is not available in the product 'sentinel-1:GRD'

Using the `dl.raster.ndarray` parameters, we can resample and align disparate datasets to the same resolution, projection, and grid alignment. Each of the resulting arrays thus have the same shape and size.

In [ ]:
print('Sentinel-1 data shape: {}'.format(all_stats[0].shape))
print('Sentinel-2 data shape: {}'.format(all_stats[1].shape))

In [ ]:
# combine the stats into a single array
all_stats = np.dstack(all_stats).data

fig = plt.figure(figsize=(10,10))
labels = ['Sentinel-1 Min VV', 
         'Sentinel-1 Min VH', 
         'Sentinel-2 Max NDVI',
         'Sentinel-2 Max NDWI']

for i in range(4):
    
    ax = plt.subplot(2, 2, i + 1)
    ax.imshow(all_stats[:, :, i])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(labels[i])
    
plt.tight_layout()

#### Build a simple decision rule based on image threshold

In [ ]:
mask = np.where(((all_stats[:, :, 0] < 60) & (all_stats[:, :, 1] < 60)
         & (all_stats[:, :, 2] < 35000) & (all_stats[:, :, 3] > 35000)), 1, 0)

fig = plt.figure(figsize=(6,6))
plt.imshow(mask, cmap='Blues')
plt.xticks([])
plt.yticks([])
plt.tight_layout()

#### Build a simple, supervised machine learning model

Different ways obtain training data using the Descartes Labs Platform:
1. Load existing reference information and extract image data
2. Use an existing Platform product (e.g. Cropland Data Layer) to sample reference data
3. Digitize reference data using Viewer

In [ ]:
with open('water_train.json') as f:
    train_data = json.load(f)

In [ ]:
train_features = [[feature['properties']['vv'],
                  feature['properties']['vh'],
                  feature['properties']['ndvi'],
                  feature['properties']['ndwi']] for feature in train_data['features']]

labels = [feature['properties']['class'] for feature in train_data['features']]

le = sklearn.preprocessing.LabelEncoder()
labels = le.fit_transform(labels)

clf = sklearn.ensemble.RandomForestClassifier(n_estimators=10, 
                                              criterion='gini', 
                                              min_samples_split=2, 
                                              min_samples_leaf=1, 
                                              min_weight_fraction_leaf=0.0, 
                                              max_features='auto', 
                                              bootstrap=True, 
                                              n_jobs=1)

clf.fit(train_features, labels)

print('feature importances\n')
for fi, label in zip(clf.feature_importances_, ['vv', 'vh', 'ndvi', 'ndwi']): 
    print("{:<10}: {:5f}".format(label, fi))

##### Save the classifier to storage

In [ ]:
from descarteslabs.ext.storage import Storage
storage_client = Storage()

classifier_key = 'water_demo_classifier'
storage_client.set(classifier_key, pickle.dumps(clf))

##### Test predictions

In [ ]:
features = all_stats.reshape(all_stats.shape[0]*all_stats.shape[1], all_stats.shape[2])
predictions = clf.predict(features)
predictions = predictions.reshape(all_stats.shape[0], all_stats.shape[1])

fig = plt.figure(figsize=(6,6))
plt.imshow(predictions, cmap='Blues')
plt.xticks([])
plt.yticks([])
plt.tight_layout()

### Scaling the process

We'll now create a tiling over Bangladesh to efficiently scale the process.

Here, we will be getting imagery over Bangladesh, calculating the image features for the classification, using the clustering algorithm to identify water in the image, and saving the resulting water mask to a new product cataloged in the Descartes Labs Platform.

First, we'll create a new product for the water mask results.

In [ ]:
''' DL Catalog to create a new product '''
from descarteslabs.ext.catalog import catalog


product_id = catalog.add_product('demo:water:bangladesh:v1',
                                  title='Water mask',
                                  description='Water mask derived from Sentinel-1 and -2 for demo'
                                  )['data']['id']

colormap = [['255', '255', '255', '255'],
            ['66', '134', '244', '255']]

band_id = catalog.add_band(product_id,  
                           'water',
                           jpx_layer=0,
                           srcfile=0,
                           srcband=1,  
                           nbits=8,
                           dtype='Byte',
                           nodata=255,
                           data_range=[0, 2**8 - 1],
                           type='class',
                           default_range=(0,255),
                           colormap=colormap)['data']['id']
print product_id

##### Define dates of interest.

In [ ]:
year = '2017'

dates = []
for month in range(3,11,2):
    str_month = str(month).zfill(2)
    last_day = calendar.monthrange(int(year), month)[1]
    dates.append(['{}-{}-{}'.format(year, str_month, '01'), '{}-{}-{}'.format(year,str_month,last_day)])

pprint(["{} - {}".format(date[0], date[1]) for date in dates])

##### Next, we create a tiling over Bangladesh.

In [ ]:
''' DLTiles to create a global tiling across our AOI '''

resolution = 20
tile_dimension = 2048
pad = 20

tiles = dl.raster.dltiles_from_shape(resolution = resolution, 
                                     tilesize = tile_dimension, 
                                     pad = pad, 
                                     shape = country)

print('Number of tiles = {}'.format(len(tiles['features'])))

##### Now, we write our function to do the classification and save the product to file. 

In [ ]:
def predict_task(tile, dates, product_dic, clf_key, product_id):
    """ predict water over dltile with Tasks """
    
    import descarteslabs as dl
    from descarteslabs.ext.catalog import Catalog
    from descarteslabs.ext.storage import Storage
    
    storage_client = Storage()
    catalog = Catalog()
    
    import os
    from osgeo import gdal, osr, ogr
    import numpy as np
    import pickle
    import sklearn
    import sklearn.ensemble
    
    def geotransform_from_dltile(dltile):
        """Returns the geotransform from a given dltile
        Inputs:
            DLTILE (dict) - dltile as returned by dl.raster.dltile()
        Returns:
            GEOTRANSFORM (tuple) - six parameter georeferencing transform as used by GDAL
        """

        geotransform = (dltile['properties']['outputBounds'][0], dltile['properties']['resolution'], 0,
                        dltile['properties']['outputBounds'][3], 0, -dltile['properties']['resolution'])

        return geotransform

    def prj_from_dltile(dltile):
        """Returns projection as WKT from dltile
        Inputs:
            DLTILE (dict) - dltile as returned by dl.raster.dltile()
        """

        prj = osr.SpatialReference()
        prj.ImportFromEPSG(int(dltile['properties']['cs_code'].split(':')[1]))

        return prj.ExportToWkt()

    def save_ds(arr, tile, outfile):
            """ save numpy array to tiff using dltile meta """

            geotransform = geotransform_from_dltile(tile)
            prj = prj_from_dltile(tile)

            driver = gdal.GetDriverByName('GTiff')
            opts = ['TILED=YES', 'COMPRESS=LZW']
            out_ds  = driver.Create(outfile, arr.shape[1], arr.shape[0], 1, gdal.GDT_Byte, options=opts)
            out_ds.SetGeoTransform(geotransform)
            out_ds.SetProjection(prj)

            out_band = out_ds.GetRasterBand(1)
            out_band.WriteArray(arr)

            out_ds = None
            arr = None

    def search_and_raster(product, product_dic, tile, date):

        # metadata search for available imagery
        available_scenes = dl.metadata.search(products=[product],
                                               geom=tile['geometry'],
                                               start_time=date[0],
                                               end_time=date[1])

        # scene ids and bands
        ids = [scene['id'] for scene in available_scenes['features']]
        bands = product_dic[product]['bands']

        # get each image as ndarray
        image_stack = None
        for i, idd in enumerate(ids):

            image, meta = dl.raster.ndarray(idd,
                                           bands = bands,
                                           dltile = tile,
                                           resampler='near')

            # initialize empty array to hold each image
            if image_stack is None:

                rx = image.shape[0]
                ry = image.shape[1]
                n_images = len(ids)
                n_bands = 2
                image_stack = np.zeros((n_images, rx, ry, n_bands))

            # mask clouds if applicable
            image_stack[i, :, :, :] = cloud_mask(image, bands)

        return image_stack

    def cloud_mask(image, bands):
        ''' apply cloud-mask if requested in bands list '''

        if 'cloud-mask' in bands:
            cloud_mask = image[:, :, 2]
            image[cloud_mask == 1] = 0

            return image[:, :, :2]
        else:
            return image

    def calc_stat(image_stack, stat):
        ''' calculate stat over time dimension'''

        if stat == 'min':
            ma = np.ma.masked_equal(image_stack, 0, copy=False)
            return np.ma.min(ma, axis=0).data
        else:
            return np.max(image_stack, axis=0)[:,:,:2]
    
    
    ''' predict water over each compositing period '''
    outdir = os.path.join(os.path.expanduser('~'), 'temp')
    if not os.path.isdir(outdir):
            os.makedirs(outdir)

    clf = pickle.loads(storage_client.get(clf_key))
    key = tile['properties']['key']
    
    for date in dates:

        outfile = '{}/water_{}_{}_{}.tif'.format(outdir, key, date[0], date[1])
        
        # get imagery and stats for each product
        all_stats = []
        for product in sorted(product_dic.keys()):

            image_stack = search_and_raster(product, product_dic, tile, date)
            stats = calc_stat(image_stack, product_dic[product]['stats'])
            all_stats.append(stats)

        # combine the stats into a single array
        all_stats = np.dstack(all_stats).astype(int)
        rx = all_stats.shape[0]
        ry = all_stats.shape[1]
        n_features = all_stats.shape[2]

        # predict and reshape
        predictions = clf.predict(all_stats.reshape(rx*ry, n_features))
        predictions = predictions.reshape(rx, ry)

        # save to product to Catalog
        save_ds(predictions, tile, outfile)
        catalog.upload_image(outfile, product_id, acquired=date[0])
        
    return key

In [ ]:
''' DL Tasks to run the job '''

from descarteslabs.client.services.tasks import AsyncTasks, as_completed
at = AsyncTasks()

async_function = at.create_function(predict_task, 
                                    name="demo-predict-water",
                                    memory='5Gi',
                                    image='us.gcr.io/dl-ci-cd/images/tasks/public/alpha/py2/default:v2018.04.26',
                                    task_timeout=1000
                                    )

In [ ]:
''' Submit DL Tasks '''

tasks = [async_function(tile, dates, product_dic, classifier_key, product_id) for tile in tiles['features']]

for task in as_completed(tasks):
    key = task.result
    print('Tile {} completed.'.format(key))